In [67]:
import pandas as pd
import json
#Garbage Collector
import gc
import pickle

from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
flightDataPath = 'gdrive/My Drive/FlightDelayPrediction/FlightData/'
weatherDataPath = 'gdrive/My Drive/FlightDelayPrediction/WeatherData/'
valid_airports = ('ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO')
years=['2016','2017']


master_df = pd.DataFrame()

# Generating Weather DataFrame

the corresponding airport id like 'JFK' has been appended to the weather data for each airport for each month in order to make the join between flight data and weather data easier. The same weather dataframe has been **saved as a pickle file** inorder to save computation during future runs of the program

In [0]:
weather_df = pd.DataFrame()

f = None
jsonOb = None

#Checks to see if previously generated pickle file is present, else generate new weather dictionary
try:
  pkl_file = open(weatherDataPath+'WeatherData.pkl','rb')
  weather_df = pickle.load(pkl_file)
  pkl_file.close()

except FileNotFoundError:
  pkl_file = open(weatherDataPath+'WeatherData.pkl','wb')
  i=False
  print('Computing!')

  for airport in valid_airports:
    for year in years:
      for month in range(1,12+1):
        f = open(weatherDataPath + airport + '/' + year + '-' + str(month)+'.json')
        jsonOb = json.load(f)
        temp_df = pd.DataFrame(jsonOb['data']['weather'])
        airportList = [airport for i in range(temp_df.shape[0])]
        temp_df['Airport']=airportList

        if i:
          weather_df = pd.concat([weather_df,temp_df],ignore_index=True)
        else:
          weather_df = temp_df
        i = True
  pickle.dump(weather_df,pkl_file)
  pkl_file.close()
        
    


| Year | Is Leap Year | No. days | No.Airports | Subtotal[No.days * No.Airports]|
|------|--------------|----------|-------------|----------|
|2016  | Yes | 366 | 15 | 5490|
|2017  | No | 365 |15 | 5475 |
||||Total|10956 entries|

In [70]:
weather_df.shape

(10965, 11)

# Joining Flight Data of All Flights with the weather data of the departure airport of the day

In [71]:
try:
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','rb')
  print('Loaded from File!')
  joined_df = pickle.load(joined_pkl)
  joined_pkl.close()

except FileNotFoundError:    
  print('Generating File!')
  joined_df = pd.DataFrame()
  temp_df = pd.DataFrame()

  i=False

  for year in years:
    for months in range(1,12+1):
      temp_df = pd.read_csv(flightDataPath + year +'/' + str(months) + '.csv',low_memory=False)
      temp_df = pd.merge(temp_df,weather_df,left_on=['FlightDate','Origin'],right_on=['date','Airport'])
      if i:
        joined_df = pd.concat([joined_df,temp_df],ignore_index=True)
      else:
        joined_df = temp_df
      i=True
  
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','wb')
  pickle.dump(joined_df,joined_pkl)
  joined_pkl.close()

Loaded from File!


In [72]:
joined_df.shape

(1877296, 122)

In [74]:
joined_df.iloc[12845]

Unnamed: 0                                                  72550
Year                                                         2016
Quarter                                                         1
Month                                                           1
DayofMonth                                                     31
                                      ...                        
totalSnow_cm                                                  0.0
date                                                   2016-01-31
astronomy       [{'moon_phase': 'Waning Gibbous', 'moonrise': ...
uvIndex                                                         0
Airport                                                       PHX
Name: 12845, Length: 122, dtype: object